In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown
import folium

import pandas as pd
import xml.etree.ElementTree as ET

import plotly.graph_objects as go

In [ ]:
%%capture

DRIVE_FOLDER = 'https://drive.google.com/drive/folders/1Jssl9iAE9OCfGWYtyMn7OR76SggJwTjd?usp=sharing'
gdown.download_folder(DRIVE_FOLDER, quiet=True)

In [ ]:
def parse_gpx(filepath):
    root = ET.parse(filepath).getroot()
    ns = '{http://www.topografix.com/GPX/1/1}'

    data = {
        'lat': [],
        'lon': [],
        'baro': [],
        'speed': [],
    }
    for trk in root.findall(f".//{ns}trk"):
        for trkseg in trk.findall(f"{ns}trkseg"):
            for trkpt in trkseg.findall(f"{ns}trkpt"):
                data['lat'].append(float(trkpt.get('lat')))
                data['lon'].append(float(trkpt.get('lon')))

                ext = trkpt.find(f"{ns}extensions")
                if ext:
                    elem = ext.find(f"{ns}baro")
                    data['baro'].append(float(elem.text) if elem is not None else None)

                    elem = ext.find(f"{ns}speed")
                    data['speed'].append(float(elem.text) if elem is not None else None)
                else:
                    data['baro'].append(None)
                    data['speed'].append(None)

    df = pd.DataFrame(data)
    return df

(
    route_df,
    polar_df,
    osmtracker_df,
) = (
    parse_gpx('standard charted hanoi marathon/route.gpx'),
    parse_gpx('standard charted hanoi marathon/polar.gpx'),
    parse_gpx('standard charted hanoi marathon/android.gpx'),
)

In [ ]:
m = folium.Map(
    location=(
      route_df['lat'].mean(),
      route_df['lon'].mean(),
    ),
    zoom_start=14,
)

def add_to_map(df, color):
    coordinates = df[['lat', 'lon']].values.tolist()
    folium.PolyLine(locations=coordinates, color=color, weight=5, opacity=1).add_to(m)

add_to_map(route_df, 'black')
add_to_map(polar_df, 'orange')
add_to_map(osmtracker_df, 'green')
m

In [ ]:
def plot_3d(df, col):
  fig = go.Figure(data=[go.Scatter3d(
      x=df['lat'],
      y=df['lon'],
      z=df[col],
      mode='markers',
      marker=dict(
          size=5,
          color=df[col],
          colorscale='Viridis',
          opacity=0.8
      )
  )])

  fig.update_layout(
      scene = dict(
          xaxis_title='Latitude',
          yaxis_title='Longitude',
          zaxis_title=col.capitalize(),
      ),
      width=700,
      margin=dict(r=20, b=10, l=10, t=40)
  )
  fig.show()

plot_3d(osmtracker_df, 'baro')

In [ ]:
plot_3d(osmtracker_df, 'speed')